In [2]:
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.tools.visualization import dag_drawer
from cutting_help_fun import *
import pickle
import networkx as nx

circ = pickle.load(open('results/supremacy_circuit_4_8.dump', 'rb' ))
original_dag = circuit_to_dag(circ)
q = circ.qregs[0]
positions = [(q[2], 1), (q[7], 1), (q[8],3), (q[9],6), (q[10], 4), (q[10],1), (q[14], 1)]
# positions = [(q[2], 1), (q[7], 1), (q[10],1), (q[14], 1)]
wires_being_cut = [x[0] for x in positions]

cut_dag, path_order_dict = cut_edges(original_dag=original_dag, positions=positions)
dag_drawer(cut_dag, filename='results/cut_dag.pdf')
print('cut_dag has %d connected components' % nx.number_weakly_connected_components(cut_dag._multi_graph))

in_out_arg_dict = contains_wire_nodes(cut_dag)
for key in in_out_arg_dict:
    print(key, in_out_arg_dict[key])
sub_reg_dicts = sub_circ_reg_counter(cut_dag, in_out_arg_dict)
# components = list(nx.weakly_connected_components(cut_dag._multi_graph))
# sub_reg_dicts, input_wires_mapping = reg_dict_counter(cut_dag, wires_being_cut)
for reg_dict in sub_reg_dicts:
    print(reg_dict)
# translation_dict = translation_dict_calc(input_wires_mapping, components, in_out_arg_dict, sub_reg_dicts)
# complete_path_map = complete_path_calc(path_order_dict, input_wires_mapping, translation_dict, sub_reg_dicts)

# [print(x, complete_path_map[x]) for x in complete_path_map]
# sub_circs = generate_sub_circs(cut_dag, positions)

cut_dag has 3 connected components
((QuantumRegister(16, 'q'), 0), 0) (True, True, True)
((QuantumRegister(16, 'q'), 0), 1) (False, False, False)
((QuantumRegister(16, 'q'), 0), 2) (False, False, False)
((QuantumRegister(16, 'q'), 1), 0) (True, True, True)
((QuantumRegister(16, 'q'), 1), 1) (False, False, False)
((QuantumRegister(16, 'q'), 1), 2) (False, False, False)
((QuantumRegister(16, 'q'), 2), 0) (False, True, True)
((QuantumRegister(16, 'q'), 2), 1) (True, False, True)
((QuantumRegister(16, 'q'), 2), 2) (False, False, False)
((QuantumRegister(16, 'q'), 3), 0) (False, False, False)
((QuantumRegister(16, 'q'), 3), 1) (True, True, True)
((QuantumRegister(16, 'q'), 3), 2) (False, False, False)
((QuantumRegister(16, 'q'), 4), 0) (True, True, True)
((QuantumRegister(16, 'q'), 4), 1) (False, False, False)
((QuantumRegister(16, 'q'), 4), 2) (False, False, False)
((QuantumRegister(16, 'q'), 5), 0) (True, True, True)
((QuantumRegister(16, 'q'), 5), 1) (False, False, False)
((QuantumRegist

In [2]:
for i, sub_circ in enumerate(sub_circs):
    dag_drawer(circuit_to_dag(sub_circ), filename='results/sub_dag_%d.pdf' % i)
    sub_circ.draw(output='text',line_length = 400, filename='results/sub_circ_%d.txt' % i)
#     pickle.dump(sub_circ, open('results/sub_circ_%d.dump' % i, 'wb' ))

print('cut_dag has %d connected components' % nx.number_weakly_connected_components(cut_dag._multi_graph))

cut_dag has 2 connected components


In [6]:
for position in positions:
    print(position)

((QuantumRegister(16, 'q'), 2), 1)
((QuantumRegister(16, 'q'), 7), 1)
((QuantumRegister(16, 'q'), 10), 1)
((QuantumRegister(16, 'q'), 14), 1)
